In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential


In [ ]:
# Generate synthetic data
np.random.seed(42)
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = X**2 + np.random.normal(0, 0.1, size=X.shape)

# Visualize the data
plt.scatter(X, y, alpha=0.6)
plt.show()


In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(50, input_dim=1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1))
    return model

def build_discriminator():
    model = Sequential()
    model.add(Dense(50, input_dim=1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    discriminator.trainable = False
    
    z = Input(shape=(1,))
    img = generator(z)
    validity = discriminator(img)
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return combined


In [ ]:
def train_gan(generator, discriminator, combined, epochs, batch_size=128):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        # ---------------------
        #  Train Discriminator
        # ---------------------
        idx = np.random.randint(0, X.shape[0], batch_size)
        real_imgs = X[idx]
        
        noise = np.random.normal(0, 1, (batch_size, 1))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------
        noise = np.random.normal(0, 1, (batch_size, 1))
        g_loss = combined.train_on_batch(noise, valid)


In [2]:
import pandas as pd

data = pd.read_csv('data/train_normalized_dataset.csv')

data.head(5)

,grupo,btsId,btsNetNome,lat_x,lon_x,cch,azimuth,ponto_id,delay_1,delay_2,delay_3,lat_y,lon_y,indoor,rssi_id,rssi_value
0,1,BTS_1_1,PEMDL01_1,-8.04342,-34.952,873,0,6604,5.0,3.0,2.0,-8.055786,-34.951505,True,rssi_1_1,0.261743
1,1,BTS_1_1,PEMDL01_1,-8.04342,-34.952,873,0,5515,5.0,3.0,2.0,-8.055784,-34.951608,True,rssi_1_1,0.228380
2,1,BTS_1_1,PEMDL01_1,-8.04342,-34.952,873,0,5049,5.0,3.0,2.0,-8.055506,-34.951693,True,rssi_1_1,0.285777
3,1,BTS_1_1,PEMDL01_1,-8.04342,-34.952,873,0,8737,4.0,7.0,2.0,-8.050390,-34.959630,False,rssi_1_1,0.581431
4,1,BTS_1_1,PEMDL01_1,-8.04342,-34.952,873,0,1203,5.0,3.0,2.0,-8.055380,-34.951736,True,rssi_1_1,0.143584


In [3]:
import pandas as pd
from copulas.multivariate import GaussianMultivariate
import matplotlib.pyplot as plt

# Load your data
data = pd.read_csv('data/train_normalized_dataset.csv')

data.drop(columns=['indoor'], inplace=True)
data.drop(columns=['rssi_id'], inplace=True)
data.drop(columns=['btsNetNome'], inplace=True)
data.drop(columns=['btsId'], inplace=True)

# Instantiate and fit the Gaussian Copula model
copula = GaussianMultivariate()
copula.fit(data)

# Sample synthetic data
synthetic_data = copula.sample(10)  # Adjust sample size as needed


# Save synthetic data
synthetic_data.to_csv('synthetic_data2.csv', index=False)


c:\Users\iantr\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
c:\Users\iantr\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:719: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
c:\Users\iantr\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
c:\Users\iantr\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:5990: RuntimeWarning: divide by zero encountered in power
  return cd2*x**(c-1)
c:\Users\iantr\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)


In [ ]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
discriminator.trainable = False

In [ ]:
generator = build_generator()
z = Input(shape=(1,))
img = generator(z)
validity = discriminator(img)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))


In [ ]:
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(new_X.ravel().reshape(-1, 1), new_y.ravel())


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
tf.random.set_seed(0)

def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_dim=latent_dim + 1),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    return model

def build_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(2,)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

optimizer_gen = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
optimizer_disc = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)

discriminator = build_discriminator()
discriminator.compile(optimizer=optimizer_disc, loss='binary_crossentropy', metrics=['accuracy'])

generator = build_generator(5)  # Latent dim

# Combine models
z = layers.Input(shape=(5,))
x_input = layers.Input(shape=(1,))
gen_y = generator(layers.concatenate([z, x_input]))  # Generator now also takes x_input
combined_input = layers.Concatenate()([x_input, gen_y])
valid = discriminator(combined_input)
combined = tf.keras.Model([x_input, z], valid)
combined.compile(optimizer=optimizer_gen, loss='binary_crossentropy')

def train(epochs, batch_size=128):
    x_data = np.linspace(-10, 10, 1000)
    y_data = 2 * x_data + 3 + np.random.normal(0, 2, x_data.shape)
    x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, x_data.shape[0], batch_size)
        x_batch = x_data[idx].reshape(-1, 1)
        y_batch = y_data[idx].reshape(-1, 1)
        noise = np.random.normal(0, 1, (batch_size, 5))
        
        gen_y_batch = generator.predict(np.hstack((noise, x_batch)))  # Provide x values as input
        d_loss_real = discriminator.train_on_batch(np.hstack((x_batch, y_batch)), valid)
        d_loss_fake = discriminator.train_on_batch(np.hstack((x_batch, gen_y_batch)), fake)
        
        g_loss = combined.train_on_batch([x_batch, noise], valid)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D loss: {0.5 * np.add(d_loss_real, d_loss_fake)[0]}, G loss: {g_loss}")

    plt.scatter(x_data, y_data, color='blue', label='Original Data')
    plt.scatter(x_data, generator.predict(np.hstack((np.random.normal(0, 1, (1000, 5)), x_data.reshape(-1, 1)))), color='red', label='Generated Data')
    plt.title("Comparison of Original and Generated Data")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()

train(epochs=300, batch_size=16)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
tf.random.set_seed(0)

def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_dim=latent_dim + 1),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    return model

def build_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(2,)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

optimizer_gen = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
optimizer_disc = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)

discriminator = build_discriminator()
discriminator.compile(optimizer=optimizer_disc, loss='binary_crossentropy', metrics=['accuracy'])

generator = build_generator(5)  # Latent dim

# Combine models
z = layers.Input(shape=(5,))
x_input = layers.Input(shape=(1,))
gen_input = layers.Concatenate()([z, x_input])
gen_y = generator(gen_input)  # Generator now also takes x_input
combined_input = layers.Concatenate()([x_input, gen_y])
valid = discriminator(combined_input)
combined = tf.keras.Model([x_input, z], valid)
combined.compile(optimizer=optimizer_gen, loss='binary_crossentropy')

def train(epochs, batch_size=128):
    x_data = np.linspace(-10, 10, 1000)
    y_data = 2 * x_data + 3 + np.random.normal(0, 2, x_data.shape)
    x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, x_data.shape[0], batch_size)
        x_batch = x_data[idx].reshape(-1, 1)
        y_batch = y_data[idx].reshape(-1, 1)
        noise = np.random.normal(0, 1, (batch_size, 5))
        
        gen_y_batch = generator.predict(np.hstack((noise, x_batch)))  # Provide x values as input
        d_loss_real = discriminator.train_on_batch(np.hstack((x_batch, y_batch)), valid)
        d_loss_fake = discriminator.train_on_batch(np.hstack((x_batch, gen_y_batch)), fake)
        
        g_loss = combined.train_on_batch([x_batch, noise], valid)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D loss: {0.5 * np.add(d_loss_real, d_loss_fake)[0]}, G loss: {g_loss}")

    plt.scatter(x_data, y_data, color='blue', label='Original Data')
    x_points = np.linspace(0, 1, 1000).reshape(-1, 1)
    noise = np.random.normal(0, 1, (1000, 5))
    generated_y = generator.predict(np.hstack((noise, x_points)))
    plt.scatter(x_points, generated_y, color='red', label='Generated Data')
    plt.title("Comparison of Original and Generated Data")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()

train(epochs=300, batch_size=128)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
tf.random.set_seed(0)

def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_dim=latent_dim + 1),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    return model

def build_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(2,)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

optimizer_gen = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
optimizer_disc = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)

discriminator = build_discriminator()
discriminator.compile(optimizer=optimizer_disc, loss='binary_crossentropy', metrics=['accuracy'])

generator = build_generator(5)  # Latent dim

# Combine models
z = layers.Input(shape=(5,))
x_input = layers.Input(shape=(1,))
gen_input = layers.Concatenate()([z, x_input])
gen_y = generator(gen_input)  # Generator now also takes x_input
combined_input = layers.Concatenate()([x_input, gen_y])
valid = discriminator(combined_input)
combined = tf.keras.Model([x_input, z], valid)
combined.compile(optimizer=optimizer_gen, loss='binary_crossentropy')

def train(epochs, batch_size=128):
    x_data = np.linspace(-10, 10, 1000)
    y_data = 2 * x_data + 3 + np.random.normal(0, 2, x_data.shape)
    x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, x_data.shape[0], batch_size)
        x_batch = x_data[idx].reshape(-1, 1)
        y_batch = y_data[idx].reshape(-1, 1)
        noise = np.random.normal(0, 1, (batch_size, 5))
        
        gen_y_batch = generator.predict(np.hstack((noise, x_batch)))  # Provide x values as input
        d_loss_real = discriminator.train_on_batch(np.hstack((x_batch, y_batch)), valid)
        d_loss_fake = discriminator.train_on_batch(np.hstack((x_batch, gen_y_batch)), fake)
        
        g_loss = combined.train_on_batch([x_batch, noise], valid)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D loss: {0.5 * np.add(d_loss_real, d_loss_fake)[0]}, G loss: {g_loss}")

    plt.scatter(x_data, y_data, color='blue', label='Original Data')
    x_points = np.linspace(0, 1, 1000).reshape(-1, 1)
    noise = np.random.normal(0, 1, (1000, 5))
    generated_y = generator.predict(np.hstack((noise, x_points)))
    plt.scatter(x_points, generated_y, color='red', label='Generated Data')
    plt.title("Comparison of Original and Generated Data")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()

train(epochs=200, batch_size=16)
